In [1]:
!pip install pyspark==3.4.2

In [2]:
!pip install minio

In [3]:
!pip install delta-spark==2.4.0

In [4]:
from pyspark.sql import SparkSession
from delta import *
from minio import Minio
import pyspark.sql.functions as F

In [5]:
spark = (
    SparkSession.builder.master("local[*]").appName("csvToParquet")
    #HADOOP
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    #DELTA
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "jars/aws-java-sdk-bundle-1.11.1026.jar, \
            jars/hadoop-aws-3.2.0.jar")
    .getOrCreate()
)


24/08/22 23:10:10 WARN Utils: Your hostname, DESKTOP-CV9TLA8 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/22 23:10:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hacs/.cache/pypoetry/virtualenvs/pyspark-delta-gPM1PAZH-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hacs/.ivy2/cache
The jars for the packages stored in: /home/hacs/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8b154350-ce3e-4221-be7d-69912f83dfd1;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 166ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [6]:
client=Minio("localhost:9000",
             access_key="minioadmin",
             secret_key="minioadmin",
             secure=False
            )


In [7]:
objects=client.list_objects('raw', prefix='CAPES/')

In [8]:
objects=client.list_objects('raw', prefix='CAPES/')
csvs = []
for object in objects:
    csvs.append(object.object_name)

In [9]:
bucket_name = 'raw'

In [75]:
def equalslist(df1, df2):
    return list(set(df1.columns).intersection(set(df2.columns)))
    
def dfjoin(dataframe1, dataframe2, columnjoin, typejoin):
    
    equals = equalslist(dataframe1,dataframe2)
    equals.remove(columnjoin)
    dataframe3 = dataframe2
    for column in equals:
        dataframe3 = dataframe3.drop(column)
        
    dataframe_join = dataframe1.join(dataframe3, columnjoin, typejoin)
    dataframe_join = dataframe_join.dropna(subset=[columnjoin])
    return dataframe_join


In [11]:
count = 0
for csv in csvs:
    print(f'{count} - {csv}')
    count+=1

0 - CAPES/2021-a-2024-autor-da-producao-intelectual-de-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
1 - CAPES/2021-a-2024-catalogo-de-teses-e-dissertacoes-brasil.csv
2 - CAPES/2021-a-2024-cursos-da-pos-graduacao-stricto-sensu-no-brasil.csv
3 - CAPES/2021-a-2024-detalhes-da-producao-intelectual-bibliografica-de-programas-de-pos-graduacao.csv
4 - CAPES/2021-a-2024-detalhes-da-producao-intelectual-tecnica-de-programas-de-pos-graduacao-stricto-sensu.csv
5 - CAPES/2021-a-2024-discentes-da-pos-graduacao-stricto-sensu-do-brasil.csv
6 - CAPES/2021-a-2024-docentes-da-pos-graduacao-stricto-sensu-no-brasil.csv
7 - CAPES/2021-a-2024-financiadores-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
8 - CAPES/2021-a-2024-membros-de-projetos-dos-programas-de-pos-graduacao-stricto-sensu-no-brasil.csv
9 - CAPES/2021-a-2024-producao-intelectual-de-pos-graduacao-stricto-sensu-no-brasil.csv
10 - CAPES/2021-a-2024-programas-da-pos-graduacao-stricto-sensu-no-brasil.csv
11 - CAPE

In [76]:
autores = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[0]}')
teses = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[1]}')
cursos = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[2]}')
bibliografica = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[3]}')
tecnica = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[4]}')
discentes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[5]}')
docentes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[6]}')
financiadores = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[7]}')
membros = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[8]}')
producoes = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[9]}')
programas = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[10]}')
projetos = spark.read.format('csv').option('header', 'true').option('sep', ';').load(f's3a://{bucket_name}/{csvs[11]}')

In [77]:
lista = equalslist(cursos, programas)
programa_curso = dfjoinouter(cursos, programas, "CD_PROGRAMA_IES", "inner")

In [25]:
print(lista)

['CD_ENTIDADE_EMEC', 'SG_UF_PROGRAMA', 'NM_REGIAO', 'DS_DEPENDENCIA_ADMINISTRATIVA', 'ID_ADD_FOTO_PROGRAMA_IES', 'CD_ENTIDADE_CAPES', 'DS_ORGANIZACAO_ACADEMICA', 'CD_PROGRAMA_IES', 'CD_AREA_AVALIACAO', 'NM_AREA_AVALIACAO', 'AN_BASE', 'NM_ENTIDADE_ENSINO', 'NM_PROGRAMA_IES', 'NM_SUBAREA_CONHECIMENTO', 'NM_MUNICIPIO_PROGRAMA_IES', 'ID_ADD_FOTO_PROGRAMA', 'CS_STATUS_JURIDICO', 'NM_ESPECIALIDADE', 'NM_GRANDE_AREA_CONHECIMENTO', 'NM_AREA_CONHECIMENTO', 'SG_ENTIDADE_ENSINO']


In [78]:
programa_curso.select("CD_ENTIDADE_CAPES", "CD_PROGRAMA_IES").show()

+-----------------+---------------+
|CD_ENTIDADE_CAPES|CD_PROGRAMA_IES|
+-----------------+---------------+
|         12002011|  12002011002P3|
|         17010004|  17010004001P4|
|         31045014|  31045014001P7|
|         33003017|  33003017034P8|
|         33051011|  33051011002P5|
|         33076014|  33076014008P9|
|         32011016|  32011016041P0|
|         40001016|  40001016061P2|
|         21001014|  21001014001P6|
|         33003017|  33003017004P1|
|         53001010|  53001010089P4|
|         31001017|  31001017110P8|
|         33072019|  33072019003P1|
|         31004016|  31004016009P4|
|         21001014|  21001014016P3|
|         40001016|  40001016065P8|
|         32004010|  32004010043P4|
|         25003011|  25003011003P2|
|         33010013|  33010013005P0|
|         30001013|  30001013030P1|
+-----------------+---------------+
only showing top 20 rows



In [79]:
projetos_membros = dfjoin(projetos, membros, "ID_PROJETO", 'inner')

In [80]:
projetos_membros.count()

1012749

In [81]:
print(f'{projetos.count()} {producoes.count()} {programas.count()}')

488718 2967 4710


In [82]:
projetos_membros.select("ID_PESSOA","NM_PROJETO", "ID_PROJETO").show()

+---------+--------------------+----------+
|ID_PESSOA|          NM_PROJETO|ID_PROJETO|
+---------+--------------------+----------+
|  3597507|AN�LISE QUIMIOM�T...|    100129|
|    15458|AN�LISE QUIMIOM�T...|    100129|
|    11306|DIN�MICA QU�NTICA...|    100140|
|  1242633|DIN�MICA QU�NTICA...|    100140|
|  3234814|DIN�MICA QU�NTICA...|    100140|
|  2521455|DIN�MICA QU�NTICA...|    100140|
|  2472203|DIN�MICA QU�NTICA...|    100140|
|    19271|PRODU��O DE NOVOS...|    100156|
|    19271|PROJETO DE INTERA...|    100160|
|   865247|AVALIA��O DE COMP...|    100191|
|  1248328|AVALIA��O DE COMP...|    100191|
|  3771900|AVALIA��O DE COMP...|    100191|
|   905075|AVALIA��O DE COMP...|    100191|
|  3099850|AVALIA��O DE COMP...|    100191|
|   499064|AVALIA��O DE COMP...|    100191|
|   439226|ESTUDOS DE QSAR P...|    100192|
|   839623|ESTUDOS DE QSAR P...|    100192|
|  1522320|ESTUDOS DE QSAR P...|    100192|
|  1178667|ESTUDOS DE QSAR P...|    100192|
|    15458|ESTUDOS DE QSAR P...|

In [85]:
projetos_membrosf = projetos_membros.dropna(subset=["ID_PROJETO"])
projetos_membrosf.count()

1012749